In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from sklearn.model_selection import train_test_split
import os
import pandas as pd

In [33]:
path = "dataset"
labelFile = 'labels.csv' 
batch_size_val = 32  
epochs_val = 10
imageDimesions = (32, 32, 3)
testRatio = 0.2    
validationRatio = 0.2 

In [4]:
images = []
classNo = []
myList = os.listdir(path)
print("Total Classes Detected:", len(myList))
noOfClasses = len(myList)

for count, folder in enumerate(myList):
    myPicList = os.listdir(os.path.join(path, folder))
    for y in myPicList:
        curImg = cv2.imread(os.path.join(path, folder, y))
        images.append(curImg)
        classNo.append(count)
    print(count, end=" ")

Total Classes Detected: 43
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 

In [7]:
images = np.array(images)
classNo = np.array(classNo)
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio, random_state=42)

In [8]:
print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Validation: {X_validation.shape}, {y_validation.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

Train: (22271, 32, 32, 3), (22271,)
Validation: (5568, 32, 32, 3), (5568,)
Test: (6960, 32, 32, 3), (6960,)


In [9]:
data = pd.read_csv(labelFile)
print("Label Data Shape:", data.shape)

Label Data Shape: (43, 2)


In [32]:
num_of_samples = []
cols = 5
num_classes = noOfClasses
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def equalize(img):
    return cv2.equalizeHist(img)

def preprocessing(img):
    img = grayscale(img)     
    img = equalize(img)      
    img = img / 255.0  
    return img

In [11]:
X_train = np.array([preprocessing(img) for img in X_train])
X_validation = np.array([preprocessing(img) for img in X_validation])
X_test = np.array([preprocessing(img) for img in X_test])

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [12]:
dataGen = ImageDataGenerator(width_shift_range=0.1,   
                            height_shift_range=0.1,
                            zoom_range=0.2,  
                            shear_range=0.1,  
                            rotation_range=10)  
dataGen.fit(X_train)

In [13]:
y_train = to_categorical(y_train, noOfClasses)
y_validation = to_categorical(y_validation, noOfClasses)
y_test = to_categorical(y_test, noOfClasses)

In [14]:
def myModel():
    model = Sequential([
        Conv2D(60, (5,5), activation='relu', input_shape=(imageDimesions[0], imageDimesions[1], 1)),
        Conv2D(60, (5,5), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(30, (3,3), activation='relu'),
        Conv2D(30, (3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        Dropout(0.5),

        Flatten(),
        Dense(500, activation='relu'),
        Dropout(0.5),
        Dense(noOfClasses, activation='softmax')  
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [23]:
model = myModel()
print(model.summary())

history = model.fit(dataGen.flow(X_train, y_train, batch_size=batch_size_val), 
                    steps_per_epoch=len(X_train)//32, 
                    epochs=epochs_val, 
                    validation_data=(X_validation, y_validation), 
                    shuffle=True)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 28, 28, 60)          │           1,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 24, 24, 60)          │          90,060 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 12, 12, 60)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 10, 10, 30)          │          16,230 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 8, 8, 30)            │           8,130 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 4, 4, 30)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 4, 4, 30)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 480)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 500)                 │         240,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 43)                  │          21,543 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 378,023 (1.44 MB)

 Trainable params: 378,023 (1.44 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 117s 152ms/step - accuracy: 0.1922 - loss: 2.9801 - val_accuracy: 0.8208 - val_loss: 0.7023
Epoch 2/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.5938 - loss: 1.4112 - val_accuracy: 0.8014 - val_loss: 0.7444
Epoch 3/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 115s 165ms/step - accuracy: 0.6368 - loss: 1.1878 - val_accuracy: 0.9382 - val_loss: 0.2338
Epoch 4/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.8125 - loss: 0.8322 - val_accuracy: 0.9352 - val_loss: 0.2318
Epoch 5/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 115s 164ms/step - accuracy: 0.7571 - loss: 0.7727 - val_accuracy: 0.9456 - val_loss: 0.1811
Epoch 6/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.8438 - loss: 0.5795 - val_accuracy: 0.9511 - val_loss: 0.1672
Epoch 7/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 131s 188ms/step - accuracy: 0.8103 - loss: 0.6252 - val_accuracy: 0.9591 - val_loss: 0.1218
Epoch 8/10
695/695 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.7812 - loss

In [28]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Score: {score[0]}')
print(f'Test Accuracy: {score[1]}')

Test Score: 0.10786592215299606
Test Accuracy: 0.9655172228813171


In [31]:
model.save("model.h5")